In [2]:
import os
import time
import requests
import PyPDF2
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:

ruta_descargas = 'C:/Users/Tomas/Desktop/script_python/polizas_argos'
ruta_controlador = 'C:/Users/Tomas/Desktop/script_python/chromedriver-win64/chromedriver.exe'

# Configurar el servicio del controlador de Chrome
service = Service(ruta_controlador)

# Opciones de navegación
opciones = Options()
prefs = {'download.default_directory': ruta_descargas}
opciones.add_experimental_option('prefs', prefs)
opciones.add_argument('--start-maximized')
opciones.add_argument('--disable-extensions')

# Crear el controlador de Chrome con las opciones y el servicio
driver = webdriver.Chrome(service=service, options=opciones)
driver.implicitly_wait(10)  # Espera implícita de 10 segundos

# Leer datos desde el archivo CSV
datos_csv = pd.read_csv('C:/Users/Tomas/Desktop/script_python/archivoCSV_ARGOS.csv')

# Abrir la página web de Argos
driver.get('https://www.argos-seguros.com/buscarPoliza.php')

# Esperar a que el elemento de automotores sea clickeable y luego hacer clic
elemento_automotores = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//a[contains(@onclick, 'hacerVisible')]"))
)
elemento_automotores.click()

# Iterar sobre las filas del DataFrame
for fila in datos_csv.itertuples(index=False):
    dni_a_ingresar = fila.DNI
    patente_a_ingresar = fila.Patente
    endoso = fila.Endoso

    # Esperar a que el campo de entrada sea visible y luego ingresarlo
    campo_dni = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.ID, "data[documento]"))
    )
    campo_patente = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.ID, "data[patente]"))
    )

    campo_dni.clear()
    campo_patente.clear()
    campo_dni.send_keys(str(dni_a_ingresar))
    campo_patente.send_keys(str(patente_a_ingresar))

    # Buscar póliza
    boton_buscar_poliza = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "submit"))
    )
    boton_buscar_poliza.click()

    try:
        # Extraer el número y nombre de póliza
        elemento_h3 = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//h3[contains(text(), 'Póliza')]"))
        )

        texto = elemento_h3.text
        numero_poliza = texto.split(' ')[1].replace(':', '')
        _, nombre_asegurado = texto.split(': ', 1)
        nombre_asegurado = nombre_asegurado.replace('/', '')

        # Construir el enlace con el número de póliza dinámico
        url_archivo = f"https://www.argos-seguros.com/verArchivo.php?&file=S3_P{numero_poliza}_E{endoso}.pdf"

        # Descargar el archivo
        nombre_archivo = f"{ruta_descargas}/{nombre_asegurado}.pdf"
        
        # Verificar si el archivo original ya existe en la carpeta de descargas
        if os.path.exists(nombre_archivo):
            # Si ya existe, buscar un nombre único agregando un número al final
            indice = 1
            while True:
                # Construir un nuevo nombre de archivo
                nuevo_nombre = f"{ruta_descargas}/{nombre_asegurado}_{indice}.pdf"
                # Verificar si el nuevo nombre ya está en uso
                if not os.path.exists(nuevo_nombre):
                    # Si el nombre no está en uso, utilizarlo y salir del bucle
                    nombre_archivo = nuevo_nombre
                    break
                # Incrementar el índice para probar el siguiente nombre
                indice += 1

        contenido_archivo = requests.get(url_archivo).content
        with open(nombre_archivo, "wb") as archivo:
            archivo.write(contenido_archivo)
            
        # Verificar el tamaño del archivo descargado (en bytes)
        tamanio_archivo_bytes = os.path.getsize(nombre_archivo)
        limite_tamanio_bytes = 2 * 1024  # Definir el límite de tamaño (2KB en bytes)

        # Verificar si el tamaño del archivo es menor al límite
        if tamanio_archivo_bytes >= limite_tamanio_bytes:
            print(f"Póliza de {nombre_asegurado} descargada exitosamente.")

            # Crear el nombre del archivo de la primera página
            nombre_archivo_frente = f"{ruta_descargas}/{nombre_asegurado}_FRENTE.pdf"

            # Leer el archivo original y copiar solo la primera página a un nuevo archivo
            with open(nombre_archivo, 'rb') as pdf_original:
                pdf_reader = PyPDF2.PdfReader(pdf_original)
                primera_pagina = pdf_reader.pages[0]
                escritor_temp = PyPDF2.PdfWriter()
                escritor_temp.add_page(primera_pagina)
                with open(nombre_archivo_frente, 'wb') as archivo_frente:
                    escritor_temp.write(archivo_frente)
            
            # Agregar una pausa para asegurarse de que el archivo esté completamente creado
            time.sleep(1)

            # Imprimir el archivo modificado dos veces
            for _ in range(2):
                os.startfile(nombre_archivo_frente, "print")
                
            # Agregar una pausa para asegurarse de que el archivo esté completamente creado
            time.sleep(5)

            # Eliminar el archivo de la primera página después de la impresión
            os.remove(nombre_archivo_frente)
        else:
            print(f"La póliza de {nombre_asegurado} no está refacturada.")
            os.remove(nombre_archivo)


    except Exception as e:
        print(f"No se pudo obtener la póliza para el asegurado. Error: {e}")


    driver.back()

driver.quit()


La póliza de BOCACCIO GUIDO no está refacturada.
Póliza de GOMEZ CLAUDIO JAVIER descargada exitosamente.
Póliza de ERMENEJILDO JONATAN RAUL descargada exitosamente.
Póliza de GOMEZ JUAN CARLOS descargada exitosamente.
Póliza de ROMANO CESAR AGUSTIN descargada exitosamente.
Póliza de RODRIGUEZ JORGE ALBERTO descargada exitosamente.
